In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

/Users/natashacarpcast/anaconda3/envs/nlp-finalproject/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("data/selfimpr_preprocessed.csv")
corpus = data["cleaned_text"].to_list()

/var/folders/5q/yq3hk8g1793ckqmn2n3xpr6c0000gn/T/ipykernel_2263/1541928078.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/selfimpr_preprocessed.csv")


In [9]:
seed_topic = [
    "morality", "moral", "immoral", "ethics", "ethical", 
    "principle", "values", "good", "right", "wrong", 
    "duty", "obligation", "responsibility",
    "ought", "should", "integrity", "character"]


In [10]:
guided_topic_model = BERTopic(seed_topic_list=seed_topic, 
            vectorizer_model=CountVectorizer(stop_words="english"))
topics, probs = guided_topic_model.fit_transform(corpus)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitl

Calculate coherence

In [13]:
def calculate_coherence(topic_model, docs, topics):
    # Preprocess documents
    cleaned_docs = topic_model._preprocess_text(docs)
    # Extract vectorizer and tokenizer from BERTopic
    vectorizer = topic_model.vectorizer_model
    tokenizer = vectorizer.build_tokenizer()
    # Extract features for Topic Coherence evaluation
    tokens = [tokenizer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    # Get topic words from BERTopic
    topic_words = []
    for topic in range(len(set(topics))-1):
        topic_words.append([word for word, _ in topic_model.get_topic(topic)])

    # Evaluate coherence
    coherence_model = CoherenceModel(topics=topic_words, 
                                    texts=tokens, 
                                    corpus=corpus,
                                    dictionary=dictionary, 
                                    coherence='c_v')
    coherence = coherence_model.get_coherence()

    return coherence

In [14]:
calculate_coherence(guided_topic_model, corpus, topics)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.505977306977948

Observe topics info

In [9]:
topic_info = guided_topic_model.get_topic_info()

In [11]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,307937,-1_im_life_feel_want,"[im, life, feel, want, like, work, know, dont,...",[i would highly recommend seeing a mental heal...
1,0,11145,0_media_instagram_phone_facebook,"[media, instagram, phone, facebook, deleted, s...",[i deleted my social media instagram tiktok sn...
2,1,8345,1_shes_girl_relationship_ex,"[shes, girl, relationship, ex, broke, told, gi...",[very good points my friend im sorry for what ...
3,2,7411,2_porn_masturbation_nofap_addiction,"[porn, masturbation, nofap, addiction, masturb...",[see focusing on not watching porn will cause ...
4,3,7032,3_meditation_thoughts_negative_mindfulness,"[meditation, thoughts, negative, mindfulness, ...",[i started with guided meditations i could fin...
...,...,...,...,...,...
1167,1166,10,1166_sexual_ventral_vagal_dorsal,"[sexual, ventral, vagal, dorsal, aroused, symp...",[i dont think thats possible because if you ha...
1168,1167,10,1167_manipulate_antisocial_borderline_contrary,"[manipulate, antisocial, borderline, contrary,...",[it can be really difficult or feel impossible...
1169,1168,10,1168_therapist_awkwardalbeit_wellourselves_film,"[therapist, awkwardalbeit, wellourselves, film...",[decided to make you a to do list google searc...
1170,1169,10,1169_gym_equipment_membership_trainers,"[gym, equipment, membership, trainers, rebuild...",[ask a staff member if any of the machines con...


There are 1172. Let's look at the top 50

In [14]:
topic_info[0:50]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,307937,-1_im_life_feel_want,"[im, life, feel, want, like, work, know, dont,...",[i would highly recommend seeing a mental heal...
1,0,11145,0_media_instagram_phone_facebook,"[media, instagram, phone, facebook, deleted, s...",[i deleted my social media instagram tiktok sn...
2,1,8345,1_shes_girl_relationship_ex,"[shes, girl, relationship, ex, broke, told, gi...",[very good points my friend im sorry for what ...
3,2,7411,2_porn_masturbation_nofap_addiction,"[porn, masturbation, nofap, addiction, masturb...",[see focusing on not watching porn will cause ...
4,3,7032,3_meditation_thoughts_negative_mindfulness,"[meditation, thoughts, negative, mindfulness, ...",[i started with guided meditations i could fin...
5,4,3747,4_games_gaming_video_game,"[games, gaming, video, game, playing, play, vi...",[ive had this thought for a while and as i am ...
6,5,3626,5_friends_groups_friendships_meetup,"[friends, groups, friendships, meetup, meet, g...",[im in my too i join facebook groups for girls...
7,6,3621,6_book_books_reading_read,"[book, books, reading, read, fiction, pages, s...",[ever since high school i feel that i read muc...
8,7,3213,7_ugly_attractive_looks_beauty,"[ugly, attractive, looks, beauty, beautiful, a...",[i know that beauty privilege exists but if yo...
9,8,3126,8_therapist_therapy_therapists_insurance,"[therapist, therapy, therapists, insurance, se...",[do what your therapist tells you to do and do...


No signs of a specific topic around the seeded words in the first 50 topics. Look for most similar topic to the word "moral"

In [33]:
similar_topics, similarity = guided_topic_model.find_topics("moral", top_n = 5)

In [35]:
guided_topic_model.get_topic(similar_topics[0])

[('evil', 0.043844669219599765),
 ('bad', 0.00959275731355833),
 ('hitler', 0.00877325354114407),
 ('good', 0.005994265199058436),
 ('person', 0.005788240514537802),
 ('villain', 0.005783464857216815),
 ('hero', 0.004588625379465589),
 ('sheep', 0.004567358837237736),
 ('world', 0.004482185721327058),
 ('innocence', 0.004209675328505251)]

Look at specific documents within that topic

In [48]:
target_topic = similar_topics[0]

# Get each document's topic assignation
doc_info = guided_topic_model.get_document_info(corpus)

# Show documents assigned to that topic
docs_under_topic = doc_info[doc_info['Topic'] == target_topic]

with pd.option_context("max_colwidth", None):
    display(docs_under_topic[["Document", "Probability"]].sort_values("Probability", ascending=False)[0:50])

,Document,Probability
323807,hey thank you for the reply i had to look up lizzie velasquez these are good questions and i dont know the answer personally the way i think about it is would i feel worthy if i was lizzie velasquez had down syndrome zach efron etc i can measure those people against my standard for myself and determine if i would feel worthy in their position now im not saying my standard is the right one but its the one i use for me not for others the interesting thing to me is that a lot of these people you mention while having some serious disadvantages hawking velasquez have done a lot of noteworthy things a more interesting question to me is is a serial killer worthy is someone who has done terrible things to others worthy,1.0
342746,first of all stop calling yourself evil now and i thought there was some reason behind you doing something like the evil type like someone cheating you misuse or something like that find the reason first and overcome it accordingly but you are not alone who faces something like this so relax it takes some time or some more time have a good day,1.0
401883,good people coming from places of real power are almost always givers they come from a place of surplus and wanting to share that with others around them dont feel bad instead protect women take care of your friends listen to them help your community be a good guy to be around,1.0
390596,are you good or are you just well behaved a good person self actualizes and seeks to live an honest life following their dreams a good person tells people the harsh truths some exceptions may apply you get the gist of it if your actions are truly your own and not out of fear of rejection than continue on but if youre a slave to fear then youre neither nice nor good,1.0
390441,if youre going to be a genuinely good person youve got to be careful who you are good to the liars cheats the selfish and proud the malicious the resentful the leeches all have to be avoided the benefits of being a good person are that you get to live with yourself knowing that youve made choices that abide by your beliefs of what a human should do be what you want to see in the world but only to those who wont take advantage of you,1.0
390396,well thats up to you but generally good people are picked up by the people they are good to building a community around yourself is important as people will help you when you are down if you help them when they are down if you want to be a bad person then thats fine but your circle will be small and filled with like minded people,1.0
388939,i recommend you read the book evil the science behind humanitys dark side by dr julia shaw i saw her speak live about this book and what youre proposing is the problem this society needs solving edit she wrote nietzches quote in one of the introductory pages he who fights with monsters should look to it that he himself does not become a monster,1.0
378179,i dont understand what you mean are you saying good people should understand that they wont be paid back equal to what they give other is that what yoy mean by accept an unjust life or are you saying people who consider themselves to be good will pridefully consider themselves above the law how do you deterine what is good how do you define what is evil,1.0
366750,i honestly dont think anything is the matter with you no one needs to know how you think or your thought patterns and tbh i get the feeling that you are most likely a really good person it would seem that the way you view being a good person is only going to make you a better person like whats the matter with constantly judging yourself against others and wanting to be a better and better person,1.0
363814,people thought of me as that girl and i also had the same problems you have at least some of them and i am absolutely not that person anymore the good isnt as good as it was but the bad also isnt that bad op should learn that people change through life and nothing stays the same,1.0


Save

In [22]:
guided_topic_model.save("bertopic_model")

2025-04-20 11:40:36,161 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
